In [164]:
# import pandas, numpy and json_normalize
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

In [165]:
# To display full column width without hiding so we can see data better
pd.set_option('display.max_colwidth', -1)

In [166]:
# Load the JSON file as a string 
world_bank_str=json.load((open('data/world_bank_projects.json')))

In [167]:
# Use normalize to create table from the nested elements in mjtheme_namecode column
mjtheme_country_df=json_normalize(world_bank_str, 'mjtheme_namecode',['countryname','id'])

In [168]:
# Display the first 10 entries of the table
mjtheme_country_df.head(10)
# We can see the missing values under name which are entered as empty strings

,code,name,countryname,id
0,8,Human development,Federal Democratic Republic of Ethiopia,P129828
1,11,,Federal Democratic Republic of Ethiopia,P129828
2,1,Economic management,Republic of Tunisia,P144674
3,6,Social protection and risk management,Republic of Tunisia,P144674
4,5,Trade and integration,Tuvalu,P145310
5,2,Public sector governance,Tuvalu,P145310
6,11,Environment and natural resources management,Tuvalu,P145310
7,6,Social protection and risk management,Tuvalu,P145310
8,7,Social dev/gender/inclusion,Republic of Yemen,P144665
9,7,Social dev/gender/inclusion,Republic of Yemen,P144665


In [170]:
# Replace these missing strings with nan values for later use
mjtheme_country_df['name'].replace('',np.nan,inplace=True)

In [172]:
# Display that the missing strings have been replaced by NaN values
mjtheme_country_df.head(10)

,code,name,countryname,id
0,8,Human development,Federal Democratic Republic of Ethiopia,P129828
1,11,NaN,Federal Democratic Republic of Ethiopia,P129828
2,1,Economic management,Republic of Tunisia,P144674
3,6,Social protection and risk management,Republic of Tunisia,P144674
4,5,Trade and integration,Tuvalu,P145310
5,2,Public sector governance,Tuvalu,P145310
6,11,Environment and natural resources management,Tuvalu,P145310
7,6,Social protection and risk management,Tuvalu,P145310
8,7,Social dev/gender/inclusion,Republic of Yemen,P144665
9,7,Social dev/gender/inclusion,Republic of Yemen,P144665


In [159]:
# Create a new dataframe which serves as a map between code and theme name
mjtheme_code_map=mjtheme_country_df[["code","name"]]

In [161]:
# Drop null values from rows
mjtheme_code_map_nonnull=mjtheme_code_map.dropna(axis=0,how="any")

In [163]:
# Drop all duplicates. Now we have a map between code and theme name which can be used to fill the missing values
mjtheme_code_map_nonnull.drop_duplicates()

,code,name
0,8,Human development
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
8,7,Social dev/gender/inclusion
11,4,Financial and private sector development
18,10,Rural development
53,9,Urban development


In [182]:
# Rename the columns of the map for later merge
mjtheme_code_map_nonnull.columns=["code_theme","name_theme"]

In [184]:
# Merge the above map dataframe and the mjtheme_country_df dataframe to get a theme of project column without any missing values
merge_df_nonnull=pd.merge(mjtheme_code_map_nonnull,mjtheme_country_df,left_on='code_theme',right_on='code')

In [188]:
# there are many duplicate columns after this merge so we drop them.Now we have a dataframe final_df
# with project id, country name, theme of project and corresponding code of the theme
# Note that assumption is that each unique project id identifies a project which may have one or more themes
# If the same theme is repeated under a unique project id, it is treated as one theme
final_df=merge_df_nonnull.drop(columns=["code","name"]).drop_duplicates()

In [189]:
# display first 10 elements of final_df
final_df.head(10)

,code_theme,name_theme,countryname,id
0,8,Human development,Federal Democratic Republic of Ethiopia,P129828
1,8,Human development,People's Republic of China,P127033
2,8,Human development,Republic of Madagascar,P132616
3,8,Human development,Kingdom of Cambodia,P146271
6,8,Human development,Republic of Tajikistan,P146109
8,8,Human development,Lao People's Democratic Republic,P143025
9,8,Human development,People's Republic of Angola,P122700
10,8,Human development,People's Republic of Bangladesh,P145118
11,8,Human development,Republic of Congo,P128628
12,8,Human development,Democratic Republic of Sao Tome and Prin,P098847


In [195]:
# Question 1 Find the 10 countries with most projects. 
# In final_df we can have same id (which represents 1 project) associated with many themes. 
# So we delete the name_theme and code_theme columns , remove duplicate entries and do a value counts on country_name
final_df_countryonly=final_df.drop(columns=["code_theme","name_theme"]).drop_duplicates()

In [196]:
final_df_countryonly.head(10)

,countryname,id
0,Federal Democratic Republic of Ethiopia,P129828
1,People's Republic of China,P127033
2,Republic of Madagascar,P132616
3,Kingdom of Cambodia,P146271
6,Republic of Tajikistan,P146109
8,Lao People's Democratic Republic,P143025
9,People's Republic of Angola,P122700
10,People's Republic of Bangladesh,P145118
11,Republic of Congo,P128628
12,Democratic Republic of Sao Tome and Prin,P098847


In [205]:
final_df_countryonly_reset=final_df_countryonly.reset_index(drop=True)

In [211]:
# Answer: Top 10 countries according to number of projects
final_df_countryonly_reset["countryname"].value_counts().head(10)

Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
Nepal                              12
People's Republic of Bangladesh    12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

In [212]:
# Question 2:Find the top 10 major project themes (using column 'mjtheme_namecode')
# For this we will use the final_df dataframe. 
final_df["name_theme"].value_counts().head(10)

Environment and natural resources management    157
Rural development                               148
Public sector governance                        140
Human development                               128
Financial and private sector development        119
Social protection and risk management           116
Social dev/gender/inclusion                     114
Trade and integration                           61 
Urban development                               40 
Economic management                             33 
Name: name_theme, dtype: int64

In [214]:
# Question 3: Fill missing values in project themes 
# We already have final_df dataframe which contains the missing values. We can do some more processing to report it differently
final_df.head(10)

,code_theme,name_theme,countryname,id
0,8,Human development,Federal Democratic Republic of Ethiopia,P129828
1,8,Human development,People's Republic of China,P127033
2,8,Human development,Republic of Madagascar,P132616
3,8,Human development,Kingdom of Cambodia,P146271
6,8,Human development,Republic of Tajikistan,P146109
8,8,Human development,Lao People's Democratic Republic,P143025
9,8,Human development,People's Republic of Angola,P122700
10,8,Human development,People's Republic of Bangladesh,P145118
11,8,Human development,Republic of Congo,P128628
12,8,Human development,Democratic Republic of Sao Tome and Prin,P098847


In [215]:
final_df["code_name"]=final_df.apply(lambda row : str(row.code_theme)+":"+str(row.name_theme), axis=1)

In [217]:
final_df_combined=final_df[["countryname","id","code_name"]]

In [225]:
# Below output provides country name, project id and all the unique themes ofthe project 
final_df_combined.groupby(['countryname','id'])['code_name'].apply(lambda x: ', '.join(x)).reset_index()

,countryname,id,code_name
0,Africa,P075941,"5:Trade and integration, 7:Social dev/gender/inclusion, 4:Financial and private sector development, 10:Rural development"
1,Africa,P094183,"5:Trade and integration, 10:Rural development"
2,Africa,P118213,"2:Public sector governance, 4:Financial and private sector development, 10:Rural development"
3,Africa,P120370,"8:Human development, 5:Trade and integration, 9:Urban development"
4,Africa,P123093,"5:Trade and integration, 4:Financial and private sector development"
5,Africa,P125018,"8:Human development, 10:Rural development"
6,Africa,P130174,"5:Trade and integration, 11:Environment and natural resources management, 10:Rural development, 9:Urban development"
7,Africa,P130694,"11:Environment and natural resources management, 7:Social dev/gender/inclusion"
8,Africa,P130888,11:Environment and natural resources management
9,Africa,P144902,11:Environment and natural resources management


So in summary 
1. Question 1: Top 10 countries
Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
Nepal                              12
People's Republic of Bangladesh    12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

2. Question 2: Top 10 project themes 
Environment and natural resources management    157
Rural development                               148
Public sector governance                        140
Human development                               128
Financial and private sector development        119
Social protection and risk management           116
Social dev/gender/inclusion                     114
Trade and integration                           61 
Urban development                               40 
Economic management                             33 

3. Question 3: No missing values under name 
Examine final_df_combined.groupby(['countryname','id'])['code_name'].apply(lambda x: ', '.join(x)).reset_index() in the previous step

